In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
from string import punctuation
import re
import random
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F 
import torch.utils.data
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')


from keyword_extraction import RakeKeywordExtractor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rake = RakeKeywordExtractor()
print(device)

# Class lang: Convert each word to index and convert each index to word
# WordEmbedding - this structure can be viewed as a corpus 
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 #count SOS and EOS
        
    def addSentence(self, sentence):
        if sentence:
             for word in sentence.split(' '):
                self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
#Prepare for training data
#Convert all Unicode to ASCII, normalize the word: make everything lowercase, trim most punctuations
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#     )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = ''.join(c for c in s if c not in punctuation)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

#Load data, process data into pairs(pair[0] is current sentence, pair[1] is the next sentence)
#Pair[0] is the input sentence, pair[1] is the ground truth
def loadData():
    pf = open('rock_eng_clean.csv', encoding='utf-8')
    df = pd.read_csv(pf, delimiter=',', names=['index', 'remix', 'yead', 'singer', 'type', 'lyric','keywords','scores'])
    lyrics = df['lyric'].tolist()
    keywords = df['keywords'].tolist()
    scores = df['scores'].tolist()
    print(len(keywords))
    
    pairs = []
    cnt = 0
    songs_len = len(lyrics)
    total_line = 0
    for i in range(1, len(lyrics)):
        if(i%1000==0): print(i)
#     for i in range(1, 1000):
        lyrics_lines = lyrics[i].decode('utf-8').split('\n')
        keywords_lines = keywords[i].decode('utf-8').split('\n')
        scores_lines = scores[i].decode('utf-8').split('\n')
        lyric_line_num = len(lyrics_lines)
#         keywords_line_num = len(keywords_lines)
#         in each song
        isFirstLine = True
#     sometimes in the dataset there are 49 lines of lyrics, only 1 keywords
        if(len(keywords_lines)<len(lyrics_lines)): continue
#         print(len(lyrics_lines), len(keywords_lines))
        for j in range(lyric_line_num):
            total_line+=1
            pair = []
            current_lyric = lyrics_lines[j]
            current_keyword = keywords_lines[j]
            current_score = scores_lines[j]
            if(len(current_score)>0 and round(float(current_score))>20):
                if(isFirstLine):
                    pair.append(normalizeString(current_keyword).encode('utf-8'))
                    isFirstLine = False
                else:
                    pair.append(normalizeString(current_keyword+' '+previous_lyric).encode('utf-8'))
                pair.append(normalizeString(current_lyric).encode('utf-8'))
                pairs.append(pair)
            previous_lyric = current_lyric
    print(total_line)
    print(len(pairs))
    return pairs

#Define a prepareData function to process all the input sentence pairs
def prepareData(lang):
    pairs = loadData()
    lang = Lang(lang)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        if pair: 
            lang.addSentence(pair[0])
            lang.addSentence(pair[1])
    print("Counted words:")
    print(lang.name, lang.n_words)
    return lang, pairs


lang, pairs = prepareData('lyric')
#randomly choose a pair, print pair[0] and pair[1]
i = 0
# while i<100:
#     print(random.choice(pairs))
#     i += 1

#Define Encoder and Decoder
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

import torch.utils.data
#Prepare for training data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(lang, pair[0])
    target_tensor = tensorFromSentence(lang, pair[1])
    return (input_tensor, target_tensor)


def indexesFromWord(lang, word):
    return [lang.word2index[word]]

def tensorFromWord(lang, sentence):
    indexes = indexesFromWord(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def evaluate_word(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        input_tensor = tensorFromWord(lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
       # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            #decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

def evaluate(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        input_tensor = tensorFromSentence(lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
       # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            #decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
    

/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
cuda


/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2878: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


82848
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
2563442
1868141
Read 1868141 sentence pairs
Counting words...
Counted words:
lyric 167894


In [5]:
hidden_size = 256
encoder1 = EncoderRNN(lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, lang.n_words).to(device)
encoder1.load_state_dict(torch.load('./encoder1_rock_e1'))
decoder1.load_state_dict(torch.load('./decoder1_rock_e1'))

def keyword_generator(sentence, num):
    #keyword extraction
    keywords = list(rake.sen_extract(sentence, word_num=num, incl_scores=True))
    filter_keywords = []
    for keyword in keywords:
        if(keyword[1]>=10):
            filter_keywords.append(keyword[0])
    for keyword in filter_keywords:
        try:
            lang.word2index[keyword]
        except KeyError:
            filter_keywords.remove(keyword)
    return filter_keywords

def lyric_generator(keywords, sentence):
    if keywords:
        output_words = []
        input_sen = normalizeString(sentence)
        new_input = keywords[0] + " " + input_sen
        output_word = evaluate(encoder1, decoder1, new_input)
        output_words.append(output_word)
        for i in range(1, len(keywords)):
            new_sen = keywords[i] + " " + ' '.join(output_word)
            sen = ' '.join(new_sen.split(' ')[:-1])
            output_word = evaluate(encoder1, decoder1, sen)
            output_words.append(output_word)

        # encode utf-8
        output = []
        for sen in output_words:
            output.append(list(map(lambda x: x, sen)))

        return output
    else:
        return None

def rock_generator(sentence):
    keywords = keyword_generator(sentence, 4)
    generated_lyric = lyric_generator(keywords, sentence)
    if generated_lyric:      
        key = []
        for keyword in keywords:
            key.append(keyword.encode('utf-8'))

        print("input: ")
        print(sentence)
        print()
        print("keywords:")
        print(key)
        print()
        print("output:")
        for out in generated_lyric:
            print(' '.join(out))
    else:
        print("Error: We cannot extract keywords, please provide a more meaningful sentence")

In [13]:
sentence = "I wanna touch ya, love ya, baby all night"
rock_generator(sentence)

input: 
I wanna touch ya, love ya, baby all night

keywords:
['touch', 'night', 'love', 'babi']

output:
You touch me where I can touch you <EOS>
Im all right through the night <EOS>
You left me were in the sun <EOS>
You left a hole where we can go <EOS>


In [12]:
sentence = "You can leave with me or you could have the blues"
rock_generator(sentence)

input: 
You can leave with me or you could have the blues

keywords:
['could', 'blue', 'leav']

output:
You could feel the end <EOS>
You will be the end of me <EOS>
You left a little more <EOS>


In [11]:
sentence = "today is a sunny day, I am so happy"
rock_generator(sentence)

input: 
today is a sunny day, I am so happy

keywords:
['sunni', 'happi', 'today', 'day']

output:
You are not to make it up <EOS>
You gotta get away and I feel <EOS>
Im not a way today <EOS>
Im going to make it through <EOS>


In [14]:
sentence = "ha ha ha ha ha ha"
rock_generator(sentence)

Error: We cannot extract keywords, please provide a more meaningful sentence
